In [110]:
import requests
import numpy as np
import pandas as pd
import scaleapi
import json
from datetime import datetime
import math
import sys
from scaleapi.tasks import TaskReviewStatus, TaskStatus


In [12]:
api_key="live_ed802f9607c14973a63a0b5ebefafcad" # Live one

In [10]:
import requests
from requests.auth import HTTPBasicAuth

url = "https://api.scale.com/v1/tasks"

headers = {"Accept": "application/json"}

auth = HTTPBasicAuth('test_4e8dee75fcdb4c0f95d6b40ee3ccd3e1', '') # No password

response = requests.request("GET", url, headers=headers, auth=auth)

print(response.text)

lback_url":"http://example.com/callback","response":{"annotations":{"url":"https://scaleapi-attachments.s3.amazonaws.com/54fbf7a0-82b5-11e7-9862-bbb3c9da12a1-3"},"is_customer_fix":false},"updated_at":"2020-08-06T23:52:48.500Z","work_started":false},{"task_id":"5f2c975f821f620016339fee","created_at":"2020-08-06T23:50:55.640Z","completed_at":"2020-08-06T23:50:55.864Z","type":"lidarannotation","status":"completed","instruction":"Please label all cars, pedestrians, and cyclists in each frame.","params":{"attachment_type":"json","attachments":["https://test124124.s3-ca-central-1.amazonaws.com/srnvs123/frames/frame_0.json","https://test124124.s3-ca-central-1.amazonaws.com/srnvs123/frames/frame_1.json","https://test124124.s3-ca-central-1.amazonaws.com/srnvs123/frames/frame_2.json"],"labels":["car","pedestrian","cyclist"],"max_distance_meters":30,"meters_per_unit":1,"labeling_sample_rate":1},"is_test":true,"urgency":"standard","metadata":{},"processed_attachments":[],"project":"Scale AI_defaul

In [77]:


url = "https://api.scale.com/v1/projects"

authorization = api_key
headers = {
    "Accept": "application/json",
    "Authorization": authorization
}

response = requests.request("GET", url, headers=headers)

print(response.text)
# return response.json

{"status_code":401,"error":"You are not providing your API key through basic auth or your API key is malformed."}


In [78]:
client = scaleapi.ScaleClient(api_key)


In [79]:
def list_projects(client):
    project_limit = 6
    num_projects = 0
    projects = client.projects()
    for project in projects:
        counter += 1
        print(f'Downloading project {counter} | {project.name} | {project.type}')

In [105]:
def list_tasks(client, project_name="Traffic Sign Detection"):
    tasks = client.get_tasks(
        project_name = project_name
    )
    num_tasks=0
    task_ids = []
    # Iterating through the generator
    for task in tasks:
        # Download task or do something!
        num_tasks = num_tasks+1
        task_ids.append(task.task_id)
        # print(task.task_id)

    # For retrieving results as a Task list
    task_list = list(tasks)
    print(str(num_tasks)+" tasks retrieved")
    return task_ids, num_tasks
list_tasks(client)

8 tasks retrieved


(['5f127f6f26831d0010e985e5',
  '5f127f6c3a6b1000172320ad',
  '5f127f699740b80017f9b170',
  '5f127f671ab28b001762c204',
  '5f127f643a6b1000172320a5',
  '5f127f5f3a6b100017232099',
  '5f127f5ab1cb1300109e4ffc',
  '5f127f55fdc4150010e37244'],
 8)

In [81]:
flag_output = {
    "task_id" : 
}

SyntaxError: invalid syntax (<ipython-input-81-09059a9b0b14>, line 3)

In [143]:
def get_task(task_id='5f127f6f26831d0010e985e5'):
    task = client.get_task(task_id)

    print(task.status)  # Task status ("pending", "completed", "error", "canceled")
    if(task.status == "completed"):
        return task_id, task
        #do stuff like get number of 
    # print(task.response) # If task is complete
    # print(json_object)
    # print(task.response['annotations'])
    else: 
        print("Task not complete.")
        #print error here
task_id, task = get_task()

def get_num_unique_labels(task):
    tasks_df = pd.DataFrame(task.response['annotations'])
    num_unique_labels = len(pd.unique(tasks_df['label']))
    print("Num unique labels: "+str(num_unique_labels))
    return num_unique_labels
num_unique_labels = get_num_unique_labels(task)

def create_dict(task_id, task, num_unique_labels):
    output_dict = {
            "task_id" : task_id,
            "task" : task,
            "num_unique_labels": num_unique_labels
    }
    
    # print(output_dict)
    return task_id,output_dict
task_id,entry = create_dict(task_id,task,num_unique_labels)


# Average here can be changed to median, anything really to improve this tool
def create_output_dict(task_id, task, num_unique_labels, average):
    if(num_unique_labels < average):
        flag = True
    else:
        flag = False

    output_dict = {
            "task_id" : task_id,
            "average_unique_labels" : average,
            "num_unique_labels": num_unique_labels,
            "flag": flag
    }
    return output_dict

print(task_id, entry)

completed
Num unique labels: 4
5f127f6f26831d0010e985e5 {'task_id': '5f127f6f26831d0010e985e5', 'task': Task({'task_id': '5f127f6f26831d0010e985e5', 'created_at': '2020-07-18T04:49:51.129Z', 'completed_at': '2020-07-18T05:03:26.917Z', 'type': 'annotation', 'status': 'completed', 'instruction': 'Draw a tight box around each object.', 'params': {'attachment': 'https://observesign.s3-us-west-2.amazonaws.com/traffic_sign_1.jpg', 'attachment_type': 'image', 'objects_to_annotate': ['traffic_control_sign', 'construction_sign', 'information_sign', 'policy_sign', 'non_visible_face'], 'with_labels': True, 'min_width': 0, 'min_height': 0, 'examples': [], 'annotation_attributes': {'occlusion': {'description': 'What percent of the object is occluded?', 'choices': ['0%', '25%', '50%', '75%', '100%']}, 'truncation': {'description': 'What percent of the object is truncated?', 'choices': ['0%', '25%', '50%', '75%', '100%']}, 'background_color': {'description': 'What color is the background of the sign?

In [148]:
def main():
    unique_labels_dict = {}
    num_unique_labels_sum = 0
    tasks, num_tasks = list_tasks(client)

    # many ways to optimize here - either do it with moving average, median number of label types or another method like assuming normal distribution and find exptected number of label types
    for task_id in tasks:
        task_id, task = get_task(task_id=task_id)
        num_unique_labels = get_num_unique_labels(task)
        num_unique_labels_sum = num_unique_labels_sum + num_unique_labels
        task_id, value = create_dict(task_id,task,num_unique_labels)
        unique_labels_dict[task_id] = value

    average_unique_labels = math.ceil(num_unique_labels_sum/num_tasks)

    print("Average unique labels: " +str(average_unique_labels))
    output_dict = {}
    for task_id in tasks:
        output_dict[task_id] = create_output_dict(
            task_id = unique_labels_dict[task_id]['task_id'], 
            task=unique_labels_dict[task_id]['task'],            
            num_unique_labels=unique_labels_dict[task_id]['num_unique_labels'], 
            average=average_unique_labels
        )
    print(output_dict.keys())

    print("A 'True' status indicates that a task needs to be reviewed.")
    for task_id in tasks:
        print(str(task_id)+" : "+str(output_dict[task_id]['flag']))
    make_output(output_dict)

def make_output(output_dict):
    # Makes new output folder
    path = os.getcwd()
    newpath = path+"/output/"
    if not os.path.exists(newpath):
        os.makedirs(newpath)

    # Makes output json file
    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    json_object = json.dumps(output_dict, indent = 4) 
    print(json_object)
    filename =  "./output/variety_flags.json"
    with open(filename, "w") as outfile:
        json.dump(output_dict, outfile)
main()


8 tasks retrieved
completed
Num unique labels: 4
completed
Num unique labels: 4
completed
Num unique labels: 2
completed
Num unique labels: 4
completed
Num unique labels: 4
completed
Num unique labels: 5
completed
Num unique labels: 4
completed
Num unique labels: 4
Average unique labels: 4
dict_keys(['5f127f6f26831d0010e985e5', '5f127f6c3a6b1000172320ad', '5f127f699740b80017f9b170', '5f127f671ab28b001762c204', '5f127f643a6b1000172320a5', '5f127f5f3a6b100017232099', '5f127f5ab1cb1300109e4ffc', '5f127f55fdc4150010e37244'])
A 'True' status indicates that a task needs to be reviewed.
5f127f6f26831d0010e985e5 : False
5f127f6c3a6b1000172320ad : False
5f127f699740b80017f9b170 : True
5f127f671ab28b001762c204 : False
5f127f643a6b1000172320a5 : False
5f127f5f3a6b100017232099 : False
5f127f5ab1cb1300109e4ffc : False
5f127f55fdc4150010e37244 : False
{
    "5f127f6f26831d0010e985e5": {
        "task_id": "5f127f6f26831d0010e985e5",
        "average_unique_labels": 4,
        "num_unique_labels": 4,